In [2]:
%matplotlib notebook
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import os
from IPython.display import Image
import seqalign
import reveal

# Experiment 3 - Creating and aligning diploid contig representations 

We used falcon to construct an assembly of a diploid model organism, C. Elegans. This resulted in three files:
- p_ctg.fa, containing the primary contigs
- a_ctg_base.fa, containing the base contigs
- a_ctg_all.fa, containing the alternative/associated contigs

We wrote a small script that takes these three files and uses REVEAL to align the associated contigs on top of the primary/base contigs in order to construct graph structure for each contig, in which the two homologous chromosomes of the sequenced organism are described by different paths through the graph. We don't know which path corresponds to which chromosome, but, this way, at least we can account for some of the variation observed between the two chromosomes in further downstream analysis. Let's obtain these graph structures.

In [138]:
!falcon2gfa.py p_ctg.fa a_ctg_base.fa a_ctg_all.fa --align -m5

From these graphs directly we can already detect all variations between the two chromosomes. Lets do that. For now, only consider the graphs larger than 1MB, to save some time.

In [139]:
contigs=!find . -type f -size +1M -name "0000*.gfa" -exec basename \{\} \;

In [140]:
for gfa in contigs:
    !reveal bubbles {gfa} > {gfa.replace(".gfa","")}.bubbles

So now, we created a single file for eacht contig describing the heterozygous sequence variations that follow from aligning the associated contigs to the primary contig. Let's calculate some statistics on these variations. 

In [141]:
_hetsnps=[]
_hetindels=[]
_hetstruct=[]

multiallele=0
complexb=0
for gfa in contigs:
    bubblefile=gfa.replace(".gfa",".bubbles")
    with open(bubblefile) as bubbles:
        header=bubbles.readline()
        headers=header.split()
        altidx1=6
        altidx2=7
        
        for line in bubbles:
            bubble=line.split()
            alleles=bubble[5].split(',')
            
            if len(alleles)>2:
                multiallele+=1
                continue #more than 2 alleles, skip
            if 'N' in alleles:
                complexb+=1
                continue #complex bubble
            
            varlengths=[len(a) for a in alleles]
            size=max(varlengths)
            snp=set([1])==set(varlengths) and '-' not in alleles
            indel='-' in alleles
            intersectsall=True#bubble[refidx]!='-' and bubble[altidx1]!='-' and bubble[altidx2]!='-'
            
            if snp and intersectsall:
                if bubble[altidx1]!=bubble[altidx2]:
                    _hetsnps.append((bubblefile,line))
            if indel and intersectsall:
                if bubble[altidx1]!=bubble[altidx2]:
                    _hetindels.append((bubblefile,line))
                    
            if size>50 and intersectsall:
                if bubble[altidx1]!=bubble[altidx2]:
                    _hetstruct.append((bubblefile,line))

print "Detected %d heterozygous SNPs."%len(_hetsnps)
print "Detected %d heterozygous indels."%len(_hetindels)
print "Detected %d heterozygous structural variants"%len(_hetstruct)

Detected 150 heterozygous SNPs.
Detected 6200 heterozygous indels.
Detected 77 heterozygous structural variants


So, we mainly see indel variations, shorter than 50bp. Which is to be expected, considering that we did not polish the assembly and the main sequencing error in SMRT sequencing are short indels.

Now, that we have our graphs we can use REVEAL to align them to the WBcel235 haploid assembly.

Lets download the reference genome and orient all contigs/graphs larger than 1MB before aligning them.

In [61]:
!wget ftp://ftp.ensemblgenomes.org/pub/metazoa/release-31/fasta/caenorhabditis_elegans/dna/Caenorhabditis_elegans.WBcel235.31.dna.genome.fa.gz
!gzip -d Caenorhabditis_elegans.WBcel235.31.dna.genome.fa.gz

--2016-07-27 16:46:59--  ftp://ftp.ensemblgenomes.org/pub/metazoa/release-31/fasta/caenorhabditis_elegans/dna/Caenorhabditis_elegans.WBcel235.31.dna.chromosome.III.fa.gz
           => 'Caenorhabditis_elegans.WBcel235.31.dna.chromosome.III.fa.gz'
Resolving ftp.ensemblgenomes.org... 193.62.197.94
Connecting to ftp.ensemblgenomes.org|193.62.197.94|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /pub/metazoa/release-31/fasta/caenorhabditis_elegans/dna ... done.
==> SIZE Caenorhabditis_elegans.WBcel235.31.dna.chromosome.III.fa.gz ... 4125615
==> PASV ... done.    ==> RETR Caenorhabditis_elegans.WBcel235.31.dna.chromosome.III.fa.gz ... done.
Length: 4125615 (3.9M) (unauthoritative)

Caenorhabditis_eleg 100%[=====================>]   3.93M  10.1MB/s   in 0.4s   

2016-07-27 16:46:59 (10.1 MB/s) - 'Caenorhabditis_elegans.WBcel235.31.dna.chromosome.III.fa.gz' saved [4125615]



In [142]:
!reveal orient Caenorhabditis_elegans.WBcel235.31.dna.genome.fa {" ".join(contigs)}

07/31/2016 12:02:47 AM Processing 000000F.gfa
07/31/2016 12:03:55 AM Not reverse complementing 000000F.gfa
07/31/2016 12:03:55 AM Processing 000001F.gfa
07/31/2016 12:05:04 AM Reverse complementing 000001F.gfa
07/31/2016 12:05:04 AM Processing 000002F.gfa
07/31/2016 12:06:06 AM Reverse complementing 000002F.gfa
07/31/2016 12:06:06 AM Processing 000003F.gfa
07/31/2016 12:07:10 AM Not reverse complementing 000003F.gfa
07/31/2016 12:07:10 AM Processing 000004F.gfa
07/31/2016 12:08:21 AM Reverse complementing 000004F.gfa
07/31/2016 12:08:21 AM Processing 000005F.gfa
07/31/2016 12:09:26 AM Not reverse complementing 000005F.gfa
07/31/2016 12:09:26 AM Processing 000006F.gfa
07/31/2016 12:10:26 AM Reverse complementing 000006F.gfa
07/31/2016 12:10:26 AM Processing 000007F.gfa
07/31/2016 12:11:25 AM Not reverse complementing 000007F.gfa
07/31/2016 12:11:25 AM Processing 000008F.gfa
07/31/2016 12:12:25 AM Not reverse complementing 000008F.gfa
07/31/2016 12:12:25 AM Processing 000009F.gfa
07/31/2

Now that all graphs are oriented with respect to the reference genome, we can align them.

In [143]:
for gfa in contigs:
    !reveal align -m5 Caenorhabditis_elegans.WBcel235.31.dna.genome.fa {gfa}

07/31/2016 12:36:28 AM Reading graph: 000000F.gfa ...
07/31/2016 12:36:29 AM Done.
07/31/2016 12:36:29 AM Constructing index...
07/31/2016 12:36:55 AM Done.
07/31/2016 12:36:55 AM Constructing pairwise-alignment...
07/31/2016 12:37:23 AM Merging nodes...
07/31/2016 12:37:23 AM Done.
07/31/2016 12:37:23 AM Done (99.41% identity, 4645234 bases out of 4672659 aligned, 6218 nodes out of 12579 aligned).
07/31/2016 12:37:23 AM Writing graph...
07/31/2016 12:37:25 AM Done.
07/31/2016 12:37:25 AM Alignment graph written to: Caenorhabditis_elegans.WBcel235.31.dna.genome_000000F.gfa
07/31/2016 12:37:28 AM Reading graph: 000001F.gfa ...
07/31/2016 12:37:28 AM Done.
07/31/2016 12:37:28 AM Constructing index...
07/31/2016 12:37:55 AM Done.
07/31/2016 12:37:55 AM Constructing pairwise-alignment...
07/31/2016 12:38:21 AM Merging nodes...
07/31/2016 12:38:21 AM Done.
07/31/2016 12:38:21 AM Done (97.82% identity, 4118561 bases out of 4210369 aligned, 6882 nodes out of 13774 aligned).
07/31/2016 12:38:2

Most graphs aligned with pretty high identity to the reference assembly. Lets detect bubbles from the resulting graphs that now describe three paths. Two for the two chromosomes and one for the haploid WBcel235 assembly.

In [144]:
for gfa in contigs:
    !reveal bubbles Caenorhabditis_elegans.WBcel235.31.dna.genome_{gfa} -r Caenorhabditis_elegans.WBcel235.31.dna.genome.fa > {gfa.replace(".gfa",".refaligned")}.bubbles

From these bubbles, we should now be able to detect both, homozygous variants (where both chromosomes deviate from the WBcel235 assembly) and heterozygous variants (where just one of two chromosomes deviates from the reference assembly).

In [145]:
hetsnps=[]
hetindels=[]
hetstruct=[]
homsnps=[]
homindels=[]
homstruct=[]
multiallele=0
complexb=0
for gfa in contigs:
    bubblefile=gfa.replace(".gfa",".refaligned.bubbles")
    with open(bubblefile) as bubbles:
        header=bubbles.readline()
        headers=header.split()
        s1,s2,s3=(headers[6],headers[7],headers[8]) #determine which sample corresponds to which genotype
        if s1=="Caenorhabditis_elegans.WBcel235.31.dna.genome.fa":
            refidx=6
            altidx1=7
            altidx2=8
        elif s2=="Caenorhabditis_elegans.WBcel235.31.dna.genome.fa":
            refidx=7
            altidx1=6
            altidx2=8
        else:
            refidx=8
            altidx1=6
            altidx2=7
        
        for line in bubbles:
            bubble=line.split()
            alleles=bubble[5].split(',')
            
            varlengths=[len(a) for a in alleles]
            size=max(varlengths)
            snp=set([1])==set(varlengths) and '-' not in alleles and 'N' not in alleles
            indel='-' in alleles
            intersectsall=bubble[refidx]!='-' and bubble[altidx1]!='-' and bubble[altidx2]!='-'
            
            if len(alleles)>2:
                multiallele+=1
                continue #more than 2 alleles, skip
            if 'N' in alleles:
                complexb+=1
                continue #complex bubble
            
            if snp and intersectsall:
                if bubble[altidx1]!=bubble[altidx2]:
                    hetsnps.append((bubblefile,line))
                else:
                    homsnps.append((bubblefile,line))
            if indel and intersectsall:
                if bubble[altidx1]!=bubble[altidx2]:
                    hetindels.append((bubblefile,line))
                else:
                    homindels.append((bubblefile,line))
                    
            if size>50 and intersectsall:
                if bubble[altidx1]!=bubble[altidx2]:
                    hetstruct.append((bubblefile,line))
                else:
                    homstruct.append((bubblefile,line))

print "Detected %d variations with more than 2 alleles."%multiallele
print "Detected %d complex bubbles structures."%complexb

print "Detected %d heterozygous SNPs."%len(hetsnps)
print "Detected %d heterozygous indels."%len(hetindels)
print "Detected %d heterozygous structural variants wrt reference."%len(hetstruct)

print "Detected %d homozygous SNPs."%len(homsnps)
print "Detected %d homozygous indels."%len(homindels)
print "Detected %d homozygous structural variants wrt reference."%len(homstruct)

Detected 260 variations with more than 2 alleles.
Detected 82 complex bubbles structures.
Detected 109 heterozygous SNPs.
Detected 5635 heterozygous indels.
Detected 15 heterozygous structural variants wrt reference.
Detected 2220 homozygous SNPs.
Detected 92922 homozygous indels.
Detected 1259 homozygous structural variants wrt reference.


It looks like quite some complex bubble structures occured. Also, we lost some heterozygous variations that we observed in simple bubble structures from the graph based representation. We do also detect homozygous variations, but once again these are dominated by short indels. Although it seems like the approach worked, the amount of short indels is probably too high for our method to come up with proper variant calls.